<a href="https://colab.research.google.com/github/yujin-jo/mini-project-team1/blob/main/analysis_04_%EB%B0%B0%EA%B8%89%EC%82%AC%EB%B9%84%EC%A4%91%EB%B3%80%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 세팅
- 깃허브 클론 
- 드라이브 마운트 
- 파일 임포트

In [1]:
!git clone https://github.com/yujin-jo/mini-project-team1.git

Cloning into 'mini-project-team1'...
remote: Enumerating objects: 612, done.
remote: Counting objects: 100% (612/612), done.
remote: Compressing objects: 100% (574/574), done.
remote: Total 612 (delta 176), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (612/612), 1.88 MiB | 7.14 MiB/s, done.
Resolving deltas: 100% (176/176), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install plotly==5.2.1

     |████████████████████████████████| 21.8 MB 1.5 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [4]:
#기본
import pandas as pd 
import numpy as   np
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import json
#plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as io
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.validators.scatter.marker import SymbolValidator

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
movies = pd.read_csv("/content/mini-project-team1/1. Crawling/data/KOBIS_TOT (1).csv")
movies["개봉일"] = pd.to_datetime(movies["개봉일"])
movies.sample(1)

,순번,개봉연도,영화명,감독,배급사,개봉일,국적,전국스크린수,전국매출액,전국관객수,장르,등급,영화구분
1506,1507,2016,명탐정 코난: 순흑의 악몽,시즈노 코분,(주)씨제이이엔엠,2016-08-03,일본,447,3754104500,507508,애니메이션,12세이상관람가,일반영화


---
# I. 가설 설정과 분석 배경

1. 가설  :  2011년 ~ 2020년까지 국내 영화 배급사에서 CJ E&M의 비중은 연도 별로 감소했는가? 
2. 가설을 선택한 이유 : 
    - 기존에는 국내 영화의 경우 CJ E&M이 독점했왔음. 
    - 다른 신생 배급사들은 없었을까 궁금했음 
    -  기존에 있던 영세 배급사들은 성장하거나 망했을까 궁금했고, 여전히 CJ E&M이 영화 산업을 지배하고 있는지 확인하고 싶었음 

# II. 분석 방향 
1. 활용 데이터 
  - 영화 개봉연도 : 연도 별로 영화 수 집계 필요 
  - 영화 배급사 : 영화 별 배급사 정보 (KOBIS 참고) 
  - 영화 수 : 연도 별, 배급사 별로 집계한 영화 수

2. 분석 방법론 
  - 시계열 분석 활용 (2011 ~ 2020년 영화 정보) 
  - 영화 배급사의 종류가 연도 별로 어떻게 달라졌는지 확인  (중소 배급사 등 배급사가 늘었는지 확인하기 위함)
  - 개봉 연도와 제작사 별로 영화 편수를 카운트한다. 
  - 연도 별로 제작사 별 비중을 계산하여 추이를 확인한다. 

3. 시각화 
  - 비율 바 차트




# III. 분석 과정  

## 1. 데이터 테이블 overview
- 2011년 ~ 2020년까지의 총 3,375개의 영화 데이터가 들어감 
- movie 테이블에 들어간 정보를 movies.info()로 파악 
- 영화 배급사의 종류는 434개로 파악되나, 두 개 이상의 영화사가 있음 

In [6]:
# 영화 기본 정보 확인 
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3375 entries, 0 to 3374
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   순번      3375 non-null   int64         
 1   개봉연도    3375 non-null   int64         
 2   영화명     3375 non-null   object        
 3   감독      3375 non-null   object        
 4   배급사     3374 non-null   object        
 5   개봉일     3375 non-null   datetime64[ns]
 6   국적      3375 non-null   object        
 7   전국스크린수  3375 non-null   int64         
 8   전국매출액   3375 non-null   int64         
 9   전국관객수   3375 non-null   int64         
 10  장르      3375 non-null   object        
 11  등급      3375 non-null   object        
 12  영화구분    3375 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(7)
memory usage: 342.9+ KB


In [7]:
#연도 별로 몇 편의 영화가 개봉되는지 확인했다. 
movies['개봉연도'].value_counts()

2017    428
2018    402
2019    389
2016    380
2015    354
2014    346
2020    345
2013    287
2012    232
2011    212
Name: 개봉연도, dtype: int64

In [8]:
#value_count를 했을 때 하나의 영화에 두 개 이상의 배급사가 있는 것을 확인 -> 새로운 전처리가 필요하다. 
movies['배급사'].value_counts()

(주)씨제이이엔엠                                 215
롯데쇼핑㈜롯데엔터테인먼트                             165
유니버설픽쳐스인터내셔널 코리아(유)                       148
(주)넥스트엔터테인먼트월드(NEW)                       138
(주)팝엔터테인먼트                                135
                                         ... 
롯데쇼핑㈜롯데시네마                                  1
롯데쇼핑㈜롯데엔터테인먼트,타임스토리그룹(주)                    1
CGV아트하우스,그린나래미디어(주)                         1
TCO(주)더콘텐츠온,플레이리스트(주)                       1
(주)드림팩트엔터테인먼트,(주)제이앤씨미디어그룹,TCO(주)더콘텐츠온      1
Name: 배급사, Length: 434, dtype: int64

## 2. 데이터 전처리 (배급사 string 분리) 
- 배급사 string을 , 기준으로 row로 분리하여 계산 
- 분리 결과 new_movies 테이블 생성

In [9]:
new_movies = movies.assign(배급사=movies.배급사.str.split(","))
new_movies = new_movies.배급사.apply(pd.Series).merge(new_movies, right_index = True, left_index = True)
new_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3375 entries, 0 to 3374
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   0       3374 non-null   object        
 1   1       321 non-null    object        
 2   2       15 non-null     object        
 3   3       1 non-null      object        
 4   4       1 non-null      object        
 5   5       1 non-null      object        
 6   순번      3375 non-null   int64         
 7   개봉연도    3375 non-null   int64         
 8   영화명     3375 non-null   object        
 9   감독      3375 non-null   object        
 10  배급사     3374 non-null   object        
 11  개봉일     3375 non-null   datetime64[ns]
 12  국적      3375 non-null   object        
 13  전국스크린수  3375 non-null   int64         
 14  전국매출액   3375 non-null   int64         
 15  전국관객수   3375 non-null   int64         
 16  장르      3375 non-null   object        
 17  등급      3375 non-null   object        
 18  영화구분    

In [10]:
# 배급사명을 ,로 구분시키고 df화함
new_movies = movies.assign(배급사=movies.배급사.str.split(","))

# 분리된 배급사를 시리즈로 만들어준 후, 기존 테이블과 병합해줬다. 
new_movies = new_movies.배급사.apply(pd.Series).merge(new_movies, right_index = True, left_index = True)

# 분리된 배급사들을 행으로 변환하기 위해 melt함수 적용해주고, 연도 별 영화 별로 쉽게 구분할 수 있도록 sorting해주었다. 
new_movies = new_movies.melt(id_vars = ["순번", "개봉연도", "영화명", "감독", "배급사", "개봉일", "국적", "전국스크린수", "전국매출액", "전국관객수", "장르", "등급", "영화구분"], var_name="영화N구분", value_name = "배급사명분리")
new_movies = new_movies.sort_values(by = ["순번", "영화N구분"], ascending = True)

# 영화 중 분리된 배급사 값이 Null값인 경우를 제외해 주었다
new_movies = new_movies[new_movies["배급사명분리"].notnull()]

# 배급사명을 원래 배급사명에 넣었다. 
new_movies['배급사'] = new_movies["배급사명분리"]

# 불필요한 컬럼을 제거했다.
new_movies.drop(['배급사명분리', '영화N구분'], axis=1, inplace=True)

In [11]:
new_movies.info()
new_movies['개봉연도'].value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3713 entries, 0 to 3374
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   순번      3713 non-null   int64         
 1   개봉연도    3713 non-null   int64         
 2   영화명     3713 non-null   object        
 3   감독      3713 non-null   object        
 4   배급사     3713 non-null   object        
 5   개봉일     3713 non-null   datetime64[ns]
 6   국적      3713 non-null   object        
 7   전국스크린수  3713 non-null   int64         
 8   전국매출액   3713 non-null   int64         
 9   전국관객수   3713 non-null   int64         
 10  장르      3713 non-null   object        
 11  등급      3713 non-null   object        
 12  영화구분    3713 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(7)
memory usage: 406.1+ KB


2017    468
2019    447
2018    439
2016    419
2015    394
2020    388
2014    366
2013    308
2012    254
2011    230
Name: 개봉연도, dtype: int64

## 3. 데이터 분석 

### 1) 개봉 연도 별 배급사 수 파악 
- 영화 개봉을 한 배급사 수가 어떻게 달라지는지 추이를 파악해 보았다. 
- 개연도 별 배급사는 2011년 43개부터 2020년 95개까지 120% 이상 증가했다. 
- 2015년, 2018년을 제외하고 매년 증가하는 추세를 보인다. 

In [12]:
# 개봉연도 별 배급사가 만든 영화를 리스트화하여 보여줌 
movie_firm = new_movies.groupby(['개봉연도', '배급사'])['영화명'].apply(list).reset_index(name="만든영화")
movie_firm

,개봉연도,배급사,만든영화
0,2011,(주)넥스트엔터테인먼트월드(NEW),"[블라인드, 가문의 영광4 - 가문의 수난, 그대를 사랑합니다, 신들의 전쟁, 브레..."
1,2011,(주)디스테이션,[소스 코드]
2,2011,(주)미디어데이,"[명탐정 코난 : 침묵의 15분, 썬더일레븐 극장판 : 최강군단 오우거의 습격, 극..."
3,2011,(주)미로비젼,"[고백, 노다메 칸타빌레 최종악장]"
4,2011,(주)박수엔터테인먼트,"[꿀벌 하치의 대모험, 별을 쫓는 아이]"
...,...,...,...
750,2020,찬란,"[야구소녀, 찬실이는 복도 많지, 썸머 85, 환상의 마로나, 신의 은총으로, 페뷸..."
751,2020,커넥트픽쳐스(주),[봉쇄수도원 카르투시오]
752,2020,판씨네마(주),"[애프터: 그 후, 미스비헤이비어, 울프 콜, 라라걸, 트로이 디렉터스 컷, 피아니..."
753,2020,하준사,[마르지엘라]


In [13]:
# 개봉연도 별 배급사 수 카운트
movie_firm_counts = movie_firm.groupby('개봉연도')['배급사'].count().reset_index(name='배급사 수')
movie_firm_counts

,개봉연도,배급사 수
0,2011,43
1,2012,51
2,2013,61
3,2014,79
4,2015,76
5,2016,83
6,2017,92
7,2018,84
8,2019,91
9,2020,95


In [14]:
#연도별 추이를 시각화하여 보여줌 
fig = px.bar(movie_firm_counts, 
              x="개봉연도", # x축
              y="배급사 수",  # y축
              text = '배급사 수',
              title='연도 별 배급사 수 추이', # Title 
              )
fig.update_traces(texttemplate='%{text}개', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
# fig.update_traces(texttemplate=, textposition='outside')

fig.show()

### 2) 2011년 ~ 2020년 전체 기간 동안 CJ에서 배급한 영화
- CJ에서 배급한 영화는 전체 영화 중 5.95%를 차지하므로 독과점 수준은 아님을 알 수 있다. 
- CJ에서 배급한 영화 수의 추이를 보면 2013년 이후 매년 감소하며, 2011년 34편 대 2020년 10편으로 70% 이상 감소하였ㅏ. 
- CJ E&M 영화 수의 추이가 하락세를 보이는 것이 영화계 전체의 움직임인지, CJ E&M만의 특수 상황인지 파악할 필요가 있다. 이를 위해 전체 영화 배급 수를 함께 살펴볼 필요가 있다.



In [15]:
cj_all = new_movies.groupby('배급사').size().reset_index(name='배급사별영화수(연도무관)')
cj_all = cj_all.sort_values(by = "배급사별영화수(연도무관)", ascending=False)
cj_all.head(5)

,배급사,배급사별영화수(연도무관)
70,(주)씨제이이엔엠,221
161,롯데쇼핑㈜롯데엔터테인먼트,181
204,유니버설픽쳐스인터내셔널 코리아(유),149
125,(주)팝엔터테인먼트,145
9,(주)넥스트엔터테인먼트월드(NEW),145


In [16]:
fig = px.pie(cj_all, values='배급사별영화수(연도무관)', names='배급사',
             color_discrete_sequence=px.colors.sequential.RdPu_r,
             hole=.3,
             title='2011년~2020년간 제작된 영화의 배급 비중')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [17]:
cj_years = new_movies[(new_movies['배급사'].str.contains("씨제이")) | (new_movies['배급사'].str.contains("CJ", case=False))]
cj_years = cj_years.groupby('개봉연도').size().reset_index(name='CJENM배급영화수')
cj_years

,개봉연도,CJENM배급영화수
0,2011,34
1,2012,32
2,2013,33
3,2014,27
4,2015,23
5,2016,23
6,2017,21
7,2018,15
8,2019,12
9,2020,10


In [18]:
#연도별 추이를 시각화하여 보여줌 
fig = px.bar(cj_years, 
              x="개봉연도", # x축
              y="CJENM배급영화수",  # y축
              text = 'CJENM배급영화수',
              title='2011~2020 CJ ENM이 배급한 영화 수 추이', # Title 
              )
fig.update_traces(texttemplate='%{text}편', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
# fig.update_traces(texttemplate=, textposition='outside')

fig.show()

### 3) 개봉 연도 별 배급사가 제작한 영화는 얼마나 될까? 
- CJ E&M의 시장 점유율을 파악하기 위해 배급사에서 제작한 영화의 수를 파악했다. 


In [19]:
# 개봉연도 별 배급사가 만든 영화를 리스트화하여 보여줌 
movie_counts = new_movies.groupby(['개봉연도','배급사'], as_index=True)[['개봉연도','배급사']].count()


In [20]:
movie_counts = new_movies.groupby(['개봉연도','배급사']).size().reset_index(name='배급영화수')
movie_counts = movie_counts.sort_values(by = ["개봉연도", "배급영화수"], ascending=[True, False])
movie_counts.head(5)

,개봉연도,배급사,배급영화수
24,2011,롯데쇼핑㈜롯데엔터테인먼트,30
10,2011,(주)씨제이이엔엠,29
0,2011,(주)넥스트엔터테인먼트월드(NEW),18
42,2011,한국소니픽쳐스릴리징브에나비스타영화㈜,14
31,2011,워너브러더스 코리아(주),13


In [21]:
# 연도 별 개봉된 총 영화 수의 추이를 보면 2017년까지 지속적으로 증가하다 2018년부터 감소세를 보인다.
movie_counts_year = new_movies.groupby('개봉연도').size().reset_index(name='총영화수')
movie_counts_year

,개봉연도,총영화수
0,2011,230
1,2012,254
2,2013,308
3,2014,366
4,2015,394
5,2016,419
6,2017,468
7,2018,439
8,2019,447
9,2020,388


In [22]:
#연도별 추이를 시각화하여 보여줌 
fig = px.bar(movie_counts_year, 
              x="개봉연도", # x축
              y="총영화수",  # y축
              text = '총영화수',
              title='2011~2020 개봉한 영화 수 추이', # Title 
              )
fig.update_traces(texttemplate='%{text}편', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
# fig.update_traces(texttemplate=, textposition='outside')

fig.show()

In [24]:
# 왜 안되는지 너무 궁금한 부분이다.. 왜 머지 안돼???
# 머지가 적용되지 않은 이유는 'movie_counts'와 'movie_counts_year'가 string화 되었기 때문이다. merge 함수에서는 열을 쓸 때 ''를 제거해야한다. 
pd.merge('movie_counts', 'movie_counts_year', how = 'left', on='개봉연도')

TypeError: ignored

In [25]:
# 연도 별 영화사 비중을 파악하기 위해 사용자 정의함수 설정 
def ratio (a, b):
  ratio = a/b * 100
  return ratio

In [26]:
#연도 별 총 영화수 산정 
movie_counts = new_movies.groupby(['개봉연도','배급사']).size().reset_index(name='배급영화수')
movie_counts['연도별총영화수'] = movie_counts.groupby('개봉연도')['배급영화수'].transform('sum')
movie_counts

,개봉연도,배급사,배급영화수,연도별총영화수
0,2011,(주)넥스트엔터테인먼트월드(NEW),18,230
1,2011,(주)디스테이션,1,230
2,2011,(주)미디어데이,3,230
3,2011,(주)미로비젼,2,230
4,2011,(주)박수엔터테인먼트,2,230
...,...,...,...,...
750,2020,찬란,12,388
751,2020,커넥트픽쳐스(주),1,388
752,2020,판씨네마(주),7,388
753,2020,하준사,1,388


In [27]:
movie_counts['비중'] = movie_counts['배급영화수']/movie_counts['연도별총영화수']*100
movie_counts

,개봉연도,배급사,배급영화수,연도별총영화수,비중
0,2011,(주)넥스트엔터테인먼트월드(NEW),18,230,7.826087
1,2011,(주)디스테이션,1,230,0.434783
2,2011,(주)미디어데이,3,230,1.304348
3,2011,(주)미로비젼,2,230,0.869565
4,2011,(주)박수엔터테인먼트,2,230,0.869565
...,...,...,...,...,...
750,2020,찬란,12,388,3.092784
751,2020,커넥트픽쳐스(주),1,388,0.257732
752,2020,판씨네마(주),7,388,1.804124
753,2020,하준사,1,388,0.257732


In [28]:
movie_counts.sort_values(by=["개봉연도","비중"], ascending=[True, False])

,개봉연도,배급사,배급영화수,연도별총영화수,비중
24,2011,롯데쇼핑㈜롯데엔터테인먼트,30,230,13.043478
10,2011,(주)씨제이이엔엠,29,230,12.608696
0,2011,(주)넥스트엔터테인먼트월드(NEW),18,230,7.826087
42,2011,한국소니픽쳐스릴리징브에나비스타영화㈜,14,230,6.086957
31,2011,워너브러더스 코리아(주),13,230,5.652174
...,...,...,...,...,...
748,2020,주식회사 블루필름웍스,1,388,0.257732
749,2020,주식회사 풍경소리,1,388,0.257732
751,2020,커넥트픽쳐스(주),1,388,0.257732
753,2020,하준사,1,388,0.257732


In [29]:
#cj 영화도 같은 방식으로 계산 (하지만 안쓸 예정)
cj_movies = movie_counts[movie_counts['배급사'].str.contains("씨제이") | movie_counts['배급사'].str.contains("CJ", case=False)]
cj_movies['타사영화'] = cj_movies['연도별총영화수'] - cj_movies['배급영화수']

In [32]:
#연도별 배급사의 영화 수 비중 추이
movie_counts.sort_values(by="비중", ascending=False)
fig = px.bar(movie_counts, 
              x="배급영화수", # x축
              y="개봉연도",  # y축
              color = "비중",
              hover_name = "배급사",
              hover_data = ["배급영화수", "비중"],
              orientation='h',
              title='연도 별 배급사의 영화 수 및 비중 추이', # Title 
              )
fig.update_layout(barmode = "relative", xaxis={'categoryorder':'total descending'})
# fig.update_traces(texttemplate=, textposition='outside')
fig.show()

In [33]:
# 잘 안보여서 CJ, 롯데, 쇼박스, 나머지를 보려고 한다. 
# cj의 연도 별 배급 영화 수와  연도 별 총 영화 배급 수 추이 데이터를 병합해본다.
# 총 영화 수 추이 : movie_counts_year
# cj 연도 별 배급 영화 수 추이 : cj_years
# 조인키 : 연도

In [64]:
cj_years_all = pd.merge(cj_years, movie_counts_year, how='left', on='개봉연도')
cj_years_all['타사영화수'] = cj_years_all['총영화수'] - cj_years_all['CJENM배급영화수']
cj_years_all = cj_years_all[['개봉연도', 'CJENM배급영화수', '타사영화수', '총영화수']]
cj_years_all['CJ영화비중'] = cj_years_all['CJENM배급영화수'] / cj_years_all['총영화수']*100
cj_years_all['타사영화비중'] = cj_years_all['타사영화수'] / cj_years_all['총영화수']*100

In [65]:
cj_years_all

,개봉연도,CJENM배급영화수,타사영화수,총영화수,CJ영화비중,타사영화비중
0,2011,34,196,230,14.782609,85.217391
1,2012,32,222,254,12.598425,87.401575
2,2013,33,275,308,10.714286,89.285714
3,2014,27,339,366,7.377049,92.622951
4,2015,23,371,394,5.837563,94.162437
5,2016,23,396,419,5.489260,94.510740
6,2017,21,447,468,4.487179,95.512821
7,2018,15,424,439,3.416856,96.583144
8,2019,12,435,447,2.684564,97.315436
9,2020,10,378,388,2.577320,97.422680


In [70]:
# 비중으로만 계산해보자! 
#연도별 CJENM의 영화 수 비중 추이 (실패)
cj_years_all.sort_values(by="개봉연도", ascending=False)
fig = px.bar(cj_years_all, 
              x=["CJENM배급영화수", "타사영화수"], # x축
              y="개봉연도",  # y축
              orientation = 'h',
              title='연도 별 배급사의 영화 수 및 비중 추이', # Title 
              
              )

# fig.update_traces(texttemplate=, textposition='outside')
fig.show()